This iPyNotebook will run the dilutions and the IPTG induction of cells for the tRNA depletion experiment.

In [17]:
import opentrons.simulate
import os
from rich import print
import random

protocol_file = 'complete_protocol.py'
pre_data = 'day_2_pre.csv'
iptg_volume = 6 # In microliters. 6ul of 0.00625M IPTG is 0.01875mM IPTG in 200ul total

First, we'll define the locations of each cell on the plate. 

In [18]:
cell_locations = {
    'B2': ['MCH10', 'RBS0.125'],
    'C2': ['MCH25', 'RBS0.125'],
    'D2': ['MCH50', 'RBS0.125'],
    'E2': ['MCH75', 'RBS0.125'],

    'B3': ['MCH10', 'RBS0.25'],
    'C3': ['MCH25', 'RBS0.25'],
    'D3': ['MCH50', 'RBS0.25'],
    'E3': ['MCH75', 'RBS0.25'],

    'B4': ['MCH10', 'RBS0.5'],
    'C4': ['MCH25', 'RBS0.5'],
    'D4': ['MCH50', 'RBS0.5'],
    'E4': ['MCH75', 'RBS0.5'],

    'B5': ['MCH10', 'RBS1'],
    'C5': ['MCH25', 'RBS1'],
    'D5': ['MCH50', 'RBS1'],
    'E5': ['MCH75', 'RBS1'],

    'B6': ['MCH10', 'RBS2'],
    'C6': ['MCH25', 'RBS2'],
    'D6': ['MCH50', 'RBS2'],
    'E6': ['MCH75', 'RBS2'],

    'B7': ['MCH10', 'RBS4'],
    'C7': ['MCH25', 'RBS4'],
    'D7': ['MCH50', 'RBS4'],
    'E7': ['MCH75', 'RBS4'],

    'B8': ['blank', 'blank'],
    'C8': ['blank', 'blank'],
    'D8': ['blank', 'blank'],
    'E8': ['blank', 'blank'],
}

Now, we'll assign each preconditioning well to two experimental wells.

In [19]:
import random
random.seed('day 2 mch')

valid_rows = list("BCDEFG")
valid_columns = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11']

valid_wells = [f"{row}{column}" for row in valid_rows for column in valid_columns]

wells_with_cells = [well for well in cell_locations if cell_locations[well] != ['blank', 'blank']]

final_positions = {}

for well in wells_with_cells:
    position_1 = random.choice(valid_wells)
    position_1_index = valid_wells.index(position_1)
    del(valid_wells[position_1_index])
    position_2 = random.choice(valid_wells)
    position_2_index = valid_wells.index(position_2)
    del(valid_wells[position_2_index])

    final_positions[position_1] = [well, cell_locations[well][0], cell_locations[well][1]]
    final_positions[position_2] = [well, cell_locations[well][0], cell_locations[well][1]]

# Fill the rest of the valid positions with blanks

for well in valid_wells:
    final_positions[well] = final_positions.get(well, ['blank', 'blank', 'blank'])


Next, we'll calculate the appropriate well dilutions for the experiment using the data from the plate reader.

In [20]:
import pandas as pd

def parse_platereader(filename):
    """Parse the output of a Teccan plate reader and make the data tidy"""
    with open(filename, 'r') as file:
        lines = file.readlines()
    lines = [line.strip() for line in lines]
    dataframe = pd.DataFrame(columns=['time', 'well', 'label', 'value', 'temperature'])
    datasets = []
    reading_dataset = [None, None]
    start_time = None
    for i, line in enumerate(lines):
        line = line.split(',')
        if line[0].startswith('End Time:'):
            continue
        elif line[0].startswith('Start Time:'):
            start_time = i
        elif start_time and any(line) > 0 and not any(reading_dataset):
            reading_dataset[0] = i
        elif start_time and any(line) == 0 and any(reading_dataset):
            reading_dataset[1] = i # Exclusive
            datasets.append(reading_dataset)
            reading_dataset = [None, None]

    def filter_function(test_value):
        if test_value == "":
            return False
        else: 
            return True

    for dataset in datasets:
        data = lines[dataset[0]:dataset[1]]
        label = data.pop(0).split(',')[0]

        if data[0].startswith('Cycle'):
            cycle = data.pop(0).split(',')[1:]

        times = list(filter(filter_function, data.pop(0).split(',')[1:]))
        temperature = list(filter(filter_function, data.pop(0).split(',')[1:]))
        data = list(filter(filter_function, [line.split(',') for line in data]))

        for line in data:
            well = str(line.pop(0))
            line = list(filter(filter_function, line))
            for i, value in enumerate(line):
                adding_frame = pd.DataFrame({
                    'time': float(times[i]),
                    'well': well,
                    'label': label,
                    'value': float(value),
                    'temperature': float(temperature[i]),
                    }, index=[0])


                dataframe = pd.concat([dataframe, adding_frame])

    return dataframe

platereader_file = pre_data
platereader_data = parse_platereader(platereader_file)


target_od600 = 0.0234506  # This is equivilant to 0.2 OD600 on our 1cm spectrophotometer

blank_wells = [x for x in cell_locations.keys() if 'blank' in cell_locations[x]]

cell_volumes = {}
lb_volumes = {}


# Get the latest timepoint from the platereader data
latest_timepoint = platereader_data['time'].max()

# Set the OD600 column to be a float


# Get the latest OD600 values for the blank wells
blank_od600 = platereader_data.loc[(platereader_data['time'] == latest_timepoint) & (platereader_data['well'].isin(blank_wells)) & (platereader_data['label'].isin(['OD600'])), 'value'].mean()

# Get all the wells that are used in the protocol
occupied_wells = platereader_data.loc[(platereader_data['time'] == latest_timepoint) & (platereader_data['label'].isin(['OD600'])), 'well']
# Remove the blank wells
occupied_wells = [well for well in occupied_wells if well not in blank_wells]
# Only include wells we have a key for
occupied_wells = [well for well in occupied_wells if well in cell_locations.keys()]

for start_well in occupied_wells:
    well_od600 = platereader_data.loc[(platereader_data['time'] == latest_timepoint) & (platereader_data['well'] == start_well) & (platereader_data['label'].isin(['OD600'])), 'value'].mean()
    well_od600 = well_od600 - blank_od600

    cell_volume = (target_od600 / well_od600) * 200
    if cell_volume > (200-iptg_volume):
        cell_volume = (200-iptg_volume)
    if cell_volume < 0:
        cell_volume = 0
    lb_volume = (200-iptg_volume) - cell_volume

    cell_volumes[start_well] = [start_well, cell_volume]
    lb_volumes[start_well] = lb_volume


/tmp/ipykernel_6156/2935061009.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, adding_frame])


Next, we associate the calculated cell volumes and LB volumes with the final well positions

In [21]:
for well in final_positions:
    source_well = final_positions[well][0]

    if source_well == 'blank':
        final_positions[well].append(0)
        final_positions[well].append((200-iptg_volume))
    else:
        final_positions[well].append(cell_volumes[source_well][1])
        final_positions[well].append(lb_volumes[source_well])

print(final_positions)

{
    'B9': ['B2', 'MCH10', 'RBS0.125', 19.079101845534915, 174.92089815446508],
    'G8': ['B2', 'MCH10', 'RBS0.125', 19.079101845534915, 174.92089815446508],
    'D10': ['C2', 'MCH25', 'RBS0.125', 15.929761022993219, 178.0702389770068],
    'B6': ['C2', 'MCH25', 'RBS0.125', 15.929761022993219, 178.0702389770068],
    'D5': ['D2', 'MCH50', 'RBS0.125', 17.279616515662262, 176.72038348433773],
    'G10': ['D2', 'MCH50', 'RBS0.125', 17.279616515662262, 176.72038348433773],
    'B8': ['E2', 'MCH75', 'RBS0.125', 14.925993257698082, 179.07400674230192],
    'D6': ['E2', 'MCH75', 'RBS0.125', 14.925993257698082, 179.07400674230192],
    'G3': ['B3', 'MCH10', 'RBS0.25', 30.351853796258947, 163.64814620374105],
    'C11': ['B3', 'MCH10', 'RBS0.25', 30.351853796258947, 163.64814620374105],
    'F7': ['C3', 'MCH25', 'RBS0.25', 15.94058949133155, 178.05941050866846],
    'C7': ['C3', 'MCH25', 'RBS0.25', 15.94058949133155, 178.05941050866846],
    'B7': ['D3', 'MCH50', 'RBS0.25', 18.085877218289113, 175.9141227817109],
    'C10': ['D3', 'MCH50', 'RBS0.25', 18.085877218289113, 175.9141227817109],
    'G7': ['E3', 'MCH75', 'RBS0.25', 17.078270392777906, 176.9217296072221],
    'C5': ['E3', 'MCH75', 'RBS0.25', 17.078270392777906, 176.9217296072221],
    'D3': ['B4', 'MCH10', 'RBS0.5', 19.25138956410062, 174.74861043589937],
    'F2': ['B4', 'MCH10', 'RBS0.5', 19.25138956410062, 174.74861043589937],
    'B3': ['C4', 'MCH25', 'RBS0.5', 17.716951825156112, 176.28304817484388],
    'G9': ['C4', 'MCH25', 'RBS0.5', 17.716951825156112, 176.28304817484388],
    'C4': ['D4', 'MCH50', 'RBS0.5', 21.11477865862336, 172.88522134137665],
    'D4': ['D4', 'MCH50', 'RBS0.5', 21.11477865862336, 172.88522134137665],
    'F8': ['E4', 'MCH75', 'RBS0.5', 17.203558281130455, 176.79644171886954],
    'E11': ['E4', 'MCH75', 'RBS0.5', 17.203558281130455, 176.79644171886954],
    'E6': ['B5', 'MCH10', 'RBS1', 17.38852591897787, 176.61147408102212],
    'B4': ['B5', 'MCH10', 'RBS1', 17.38852591897787, 176.61147408102212],
    'G2': ['C5', 'MCH25', 'RBS1', 17.44025237658028, 176.55974762341972],
    'D7': ['C5', 'MCH25', 'RBS1', 17.44025237658028, 176.55974762341972],
    'G6': ['D5', 'MCH50', 'RBS1', 17.33710347753549, 176.6628965224645],
    'E3': ['D5', 'MCH50', 'RBS1', 17.33710347753549, 176.6628965224645],
    'C6': ['E5', 'MCH75', 'RBS1', 14.93074423959493, 179.06925576040507],
    'F3': ['E5', 'MCH75', 'RBS1', 14.93074423959493, 179.06925576040507],
    'F6': ['B6', 'MCH10', 'RBS2', 16.93031251079206, 177.06968748920795],
    'E2': ['B6', 'MCH10', 'RBS2', 16.93031251079206, 177.06968748920795],
    'D8': ['C6', 'MCH25', 'RBS2', 16.71312179333458, 177.28687820666542],
    'E9': ['C6', 'MCH25', 'RBS2', 16.71312179333458, 177.28687820666542],
    'D2': ['D6', 'MCH50', 'RBS2', 17.73705170754554, 176.26294829245447],
    'G5': ['D6', 'MCH50', 'RBS2', 17.73705170754554, 176.26294829245447],
    'E8': ['E6', 'MCH75', 'RBS2', 16.63605633327214, 177.36394366672786],
    'E4': ['E6', 'MCH75', 'RBS2', 16.63605633327214, 177.36394366672786],
    'D9': ['B7', 'MCH10', 'RBS4', 18.01640182170399, 175.983598178296],
    'C3': ['B7', 'MCH10', 'RBS4', 18.01640182170399, 175.983598178296],
    'B10': ['C7', 'MCH25', 'RBS4', 15.80628521779984, 178.19371478220017],
    'F4': ['C7', 'MCH25', 'RBS4', 15.80628521779984, 178.19371478220017],
    'D11': ['D7', 'MCH50', 'RBS4', 17.485766862414867, 176.51423313758514],
    'G11': ['D7', 'MCH50', 'RBS4', 17.485766862414867, 176.51423313758514],
    'C2': ['E7', 'MCH75', 'RBS4', 30.293040875998685, 163.70695912400132],
    'C9': ['E7', 'MCH75', 'RBS4', 30.293040875998685, 163.70695912400132],
    'B2': ['blank', 'blank', 'blank', 0, 194],
    'B5': ['blank', 'blank', 'blank', 0, 194],
    'B11': ['blank', 'blank', 'blank', 0, 194],
    'C8': ['blank', 'blank', 'blank', 0, 194],
    'E5': ['blank', 'blank', 'blank', 0, 194],
    'E7': ['blank', 'blank', 'blank', 0, 194],
    'E10': ['blank', 'blank', 'blank', 0, 194],
    'F5': ['blank', 'blan

Now we add the dictionaries to the base protocol and test to make sure it runs.

In [25]:
# Make a copy of base_protocol.py and name it complete_protocol.py
os.system('cp base_protocol_multichannel.py complete_protocol.py')

# Add the volumes to the complete_protocol.py file
with open('complete_protocol.py', 'a') as file:
    file.write(f'final_positions = {final_positions}\n')
    file.write(f'iptg_volume = {iptg_volume}\n')

# Simulate the protocol
opentrons.simulate.simulate(open(protocol_file))

([{'level': 0,
   'payload': {'instrument': <CustomPipette: p300_single_v1 in LEFT>,
    'location': H1 of Opentrons 96 Tip Rack 300 µL on 8,
    'text': 'Picking up tip from H1 of Opentrons 96 Tip Rack 300 µL on 8'},
   'logs': [<LogRecord: opentrons.calibration_storage.ot2.tip_length, 30, /home/croots/miniconda3/envs/opentrons/lib/python3.10/site-packages/opentrons/calibration_storage/ot2/tip_length.py, 53, "Tip length calibrations not found for None">]},
  {'level': 0,
   'payload': {'instrument': <CustomPipette: p300_single_v1 in LEFT>,
    'volume': 192.4129879699116,
    'location': Location(point=Point(x=278.94, y=133.4, z=7.289999999999999), labware=A1 of USA Scientific 12 Well Reservoir 22 mL on 6),
    'rate': 1.0,
    'text': 'Aspirating 192.4129879699116 uL from A1 of USA Scientific 12 Well Reservoir 22 mL on 6 at 150.0 uL/sec'},
   'logs': []},
  {'level': 0,
   'payload': {'instrument': <CustomPipette: p300_single_v1 in LEFT>,
    'volume': 174.92089815446508,
    'locati